# Prompt Evaluation and Tracing with MLflow
This notebook is designed to help you evaluate and trace prompts using MLflow. It provides a structured way to log prompt evaluations, track their performance, and visualize the results over time.

Key Benefits of MLflow Prompt Evaluation
* Effective Evaluation: `MLflow's LLM Evaluation API provides a simple and consistent way to evaluate prompts across different models and datasets without writing boilerplate code.
* Compare Results: Compare evaluation results with ease in the MLflow UI.
* Tracking Results: Track evaluation results in MLflow Experiment to maintain the history of prompt performance and different evaluation settings.
* Tracing: Inspect model behavior during inference deeply with traces generated during evaluation.

### <span style="color:red">Additions for .env - Environement Variables</span>




OPENAI_API_TYPE="azure"

OPENAI_API_BASE="<your-azure-openai-endpoint>"

OPENAI_DEPLOYMENT_NAME=gpt4o

OPENAI_API_VERSION="2024-08-01-preview"

OPENAI_API_KEY="<your-api-key-for-azure-openai>"

In [1]:
#!pip install mlflow openai dspy litellm textstat evaluate --quiet

In [1]:
import os
import mlflow
from openai import AzureOpenAI
mlflow.set_tracking_uri("http://20.75.92.162:5000/")
print(mlflow.__version__)

C:\Users\Anshu Pandey\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


3.1.1


### 1. Create a new prompt

In [2]:
import mlflow

# Use double curly braces for variables in the template
initial_template = """\
Summarize content you are provided with in {{ num_sentences }} sentences.

Sentences: {{ sentences }}
"""

# Register a new prompt
prompt = mlflow.register_prompt(
    name="anshu-summarization-prompt",
    template=initial_template,
    # Optional: Provide a commit message to describe the changes
    commit_message="Initial commit",
)

# The prompt object contains information about the registered prompt
print(f"Created prompt '{prompt.name}' (version {prompt.version})")

C:\Users\Anshu Pandey\AppData\Local\Temp\ipykernel_244784\2441474764.py:11: FutureWarning: The `mlflow.register_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.register_prompt` instead. The original API will be removed in the future release.
  prompt = mlflow.register_prompt(
2025/09/07 08:30:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: anshu-summarization-prompt, version 9


Created prompt 'anshu-summarization-prompt' (version 9)


### 2. Prepare Evaluation Data

In [3]:
import pandas as pd

eval_data = pd.DataFrame(
    {
        "inputs": [
            "Artificial intelligence has transformed how businesses operate in the 21st century. Companies are leveraging AI for everything from customer service to supply chain optimization. The technology enables automation of routine tasks, freeing human workers for more creative endeavors. However, concerns about job displacement and ethical implications remain significant. Many experts argue that AI will ultimately create more jobs than it eliminates, though the transition may be challenging.",
            "Climate change continues to affect ecosystems worldwide at an alarming rate. Rising global temperatures have led to more frequent extreme weather events including hurricanes, floods, and wildfires. Polar ice caps are melting faster than predicted, contributing to sea level rise that threatens coastal communities. Scientists warn that without immediate and dramatic reductions in greenhouse gas emissions, many of these changes may become irreversible. International cooperation remains essential but politically challenging.",
            "The human genome project was completed in 2003 after 13 years of international collaborative research. It successfully mapped all of the genes of the human genome, approximately 20,000-25,000 genes in total. The project cost nearly $3 billion but has enabled countless medical advances and spawned new fields like pharmacogenomics. The knowledge gained has dramatically improved our understanding of genetic diseases and opened pathways to personalized medicine. Today, a complete human genome can be sequenced in under a day for about $1,000.",
            "Remote work adoption accelerated dramatically during the COVID-19 pandemic. Organizations that had previously resisted flexible work arrangements were forced to implement digital collaboration tools and virtual workflows. Many companies reported surprising productivity gains, though concerns about company culture and collaboration persisted. After the pandemic, a hybrid model emerged as the preferred approach for many businesses, combining in-office and remote work. This shift has profound implications for urban planning, commercial real estate, and work-life balance.",
            "Quantum computing represents a fundamental shift in computational capability. Unlike classical computers that use bits as either 0 or 1, quantum computers use quantum bits or qubits that can exist in multiple states simultaneously. This property, known as superposition, theoretically allows quantum computers to solve certain problems exponentially faster than classical computers. Major technology companies and governments are investing billions in quantum research. Fields like cryptography, material science, and drug discovery are expected to be revolutionized once quantum computers reach practical scale.",
        ],
        "targets": [
            "AI has revolutionized business operations through automation and optimization, though ethical concerns about job displacement persist alongside predictions that AI will ultimately create more employment opportunities than it eliminates.",
            "Climate change is causing accelerating environmental damage through extreme weather events and melting ice caps, with scientists warning that without immediate reduction in greenhouse gas emissions, many changes may become irreversible.",
            "The Human Genome Project, completed in 2003, mapped approximately 20,000-25,000 human genes at a cost of $3 billion, enabling medical advances, improving understanding of genetic diseases, and establishing the foundation for personalized medicine.",
            "The COVID-19 pandemic forced widespread adoption of remote work, revealing unexpected productivity benefits despite collaboration challenges, and resulting in a hybrid work model that impacts urban planning, real estate, and work-life balance.",
            "Quantum computing uses qubits existing in multiple simultaneous states to potentially solve certain problems exponentially faster than classical computers, with major investment from tech companies and governments anticipating revolutionary applications in cryptography, materials science, and pharmaceutical research.",
        ],
    }
)

### 3. Define prediction function

In [4]:
client = AzureOpenAI()

def predict(data: pd.DataFrame) -> list[str]:
    predictions = []
    prompt = mlflow.genai.load_prompt("prompts:/anshu-summarization-prompt/1")

    for _, row in data.iterrows():
        # Fill in variables in the prompt template
        content = prompt.format(sentences=row["inputs"], num_sentences=1)
        completion = client.chat.completions.create(
            model="gpt4o",
            messages=[{"role": "user", "content": content}],
            temperature=0.1,
        )
        predictions.append(completion.choices[0].message.content)

    return predictions

### 4. Run Evaluation

In [5]:
mlflow.set_experiment('azure-prompt-evaluation')

<Experiment: artifact_location='mlflow-artifacts:/948223517203568369', creation_time=1757166166551, experiment_id='948223517203568369', last_update_time=1757166166551, lifecycle_stage='active', name='azure-prompt-evaluation', tags={}>

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:

with mlflow.start_run(run_name="anshu-prompt-evaluation"):
    mlflow.log_param("model", "gpt-4o-mini")
    mlflow.log_param("temperature", 0.1)

    results = mlflow.evaluate(
        model=predict,
        data=eval_data,
        targets="targets",
        extra_metrics=[
            mlflow.metrics.latency(),
            mlflow.metrics.genai.answer_relevance(model="openai:/gpt-4o-mini"),
            mlflow.metrics.genai.answer_similarity(model="openai:/gpt-4o-mini"),
            mlflow.metrics.ari_grade_level(),
            mlflow.metrics.flesch_kincaid_grade_level()
        ],

    )

2025/09/07 09:48:47 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/09/07 09:49:19 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

🏃 View run anshu-prompt-evaluation at: http://20.75.92.162:5000/#/experiments/948223517203568369/runs/f33c5783742d44bd93a5c1c14b2ad66b
🧪 View experiment at: http://20.75.92.162:5000/#/experiments/948223517203568369


[Trace(trace_id=ef5462988a44466899e7a7d8cc6c5efc), Trace(trace_id=1aa7bfb2cab64f66a0847431494639e5), Trace(trace_id=a58887015f5d40e08b2b2e50c76bdf94), Trace(trace_id=0fa485eb19ac43f78a15c9532ce391b4), Trace(trace_id=c30b13989be649bfb3357ec1a6066ffb)]

### 5. View Results
<img src="https://mlflow.org/docs/latest/assets/images/prompt-evaluation-result-7c106f17187fdc750439725d086c389b.png" alt="MLflow LLM Evaluation UI" width="800">

<img src = "https://mlflow.org/docs/latest/assets/images/prompt-evaluation-chart-8a93612e37184b8279c699fd6640013d.png" >

## Prompt Optimization

In [13]:
import os
from typing import Any
import mlflow
from mlflow.genai.scorers import scorer
from mlflow.genai.optimize import OptimizerConfig, LLMParams

In [14]:
# Register the initial prompt
initial_template = """
Answer to this math question: {{question}}.
Return the result in a JSON string in the format of {"answer": "xxx"}.
"""

prompt = mlflow.genai.register_prompt(
    name="anshu-math2",
    template=initial_template,
)


2025/09/07 09:50:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: anshu-math2, version 2


In [15]:

# The data can be a list of dictionaries, a pandas DataFrame, or an mlflow.genai.EvaluationDataset
# It needs to contain inputs and expectations where each row is a dictionary.
train_data = [
    {
        "inputs": {"question": "Given that $y=3$, evaluate $(1+y)^y$."},
        "expectations": {"answer": "64"},
    },
    {
        "inputs": {
            "question": "The midpoint of the line segment between $(x,y)$ and $(-9,1)$ is $(3,-5)$. Find $(x,y)$."
        },
        "expectations": {"answer": "(15,-11)"},
    },
    {
        "inputs": {
            "question": "What is the value of $b$ if $5^b + 5^b + 5^b + 5^b + 5^b = 625^{(b-1)}$? Express your answer as a common fraction."
        },
        "expectations": {"answer": "\\frac{5}{3}"},
    },
    {
        "inputs": {"question": "Evaluate the expression $a^3\\cdot a^2$ if $a= 5$."},
        "expectations": {"answer": "3125"},
    },
    {
        "inputs": {"question": "Evaluate $\\lceil 8.8 \\rceil+\\lceil -8.8 \\rceil$."},
        "expectations": {"answer": "17"},
    },
]

eval_data = [
    {
        "inputs": {
            "question": "The sum of 27 consecutive positive integers is $3^7$. What is their median?"
        },
        "expectations": {"answer": "81"},
    },
    {
        "inputs": {"question": "What is the value of $x$ if $x^2 - 10x + 25 = 0$?"},
        "expectations": {"answer": "5"},
    },
    {
        "inputs": {
            "question": "If $a\\ast b = 2a+5b-ab$, what is the value of $3\\ast10$?"
        },
        "expectations": {"answer": "26"},
    },
    {
        "inputs": {
            "question": "Given that $-4$ is a solution to $x^2 + bx -36 = 0$, what is the value of $b$?"
        },
        "expectations": {"answer": "-5"},
    },
]


In [16]:

# Define a custom scorer function to evaluate prompt performance with the @scorer decorator.
# The scorer function for optimization can take inputs, outputs, and expectations.
@scorer
def exact_match(expectations: dict[str, Any], outputs: dict[str, Any]) -> bool:
    return expectations["answer"] == outputs["answer"]



In [17]:
# Optimize the prompt
result = mlflow.genai.optimize_prompt(
    target_llm_params=LLMParams(model_name="azure/gpt4o"),
    prompt=prompt,
    train_data=train_data,
    eval_data=eval_data,
    scorers=[exact_match],
    optimizer_config=OptimizerConfig(
        num_instruction_candidates=8,
        max_few_show_examples=2,
    ),
)

# The optimized prompt is automatically registered as a new version
print(result.prompt.uri)

2025/09/07 09:50:10 INFO mlflow.genai.optimize.optimizers.dspy_mipro_optimizer: Started optimizing prompt prompts:/anshu-math2/2. Please wait as this process typically takes several minutes, but can take longer with large datasets...
2025/09/07 09:50:28 INFO mlflow.genai.optimize.optimizers.dspy_mipro_optimizer: Prompt optimization completed. Evaluation score changed from 0.0 to 25.0.
C:\Users\Anshu Pandey\AppData\Roaming\Python\Python310\site-packages\mlflow\genai\optimize\optimizers\dspy_mipro_optimizer.py:116: FutureWarning: The `mlflow.register_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.register_prompt` instead. The original API will be removed in the future release.
  optimized_prompt = register_prompt(
2025/09/07 09:50:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: anshu-math2, version 3


prompts:/anshu-math2/3


## Thank YOu